# Conversational Chat with Azure OpenAI

In the previous example, one can ask a single question and get an answer.<br><br>

In this example, we will use the Azure OpenAI library to have a conversational chat with the AI. The AI will remember the context of the conversation and respond accordingly.This is useful for building chatbots or virtual assistants that can hold a conversation with users.
***

## Prerequisites

1. Make sure that `python3` is installed on your system.
1. Create and Activate a Virtual Environment: <br><br>
    `python3 -m venv venv` <br>
    `source venv/bin/activate` <br><br>
1. Create a `.env` file in the same directory as this script and add the following variables:<br><br>
     ```
     AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>
     AZURE_OPENAI_MODEL=<your_azure_openai_model>
     AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>
     AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>
     ```
***

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [21]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


***
## Import Modules

In [22]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.
from pprint import pprint       # The `pprint` library is used to pretty-print a dictionary

## Load environment variables from .env file

The `load_dotenv()` function reads the .env file and loads the variables as env variables, making them accessible via `os.environ` or `os.getenv()`.

In [23]:
load_dotenv()

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Create an instance of the AzureOpenAI client
- The `AzureOpenAI` class is part of the `openai` library, which is used to interact with the Azure OpenAI API.
- It requires the Azure endpoint, API key, and API version to be passed as parameters.

In [24]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Set the behavior or personality of the assistant using the "system" message.

Create a global `conversation` array and initialize it with a system message. This array will hold the conversation history which will be forwarded to LLM

In [25]:
conversation=[{"role": "system", "content": "You are a sarcastic AI assistant. You are proud of your amazing memory"}]

## Call the Azure OpenAI API to get the AI's response. Append the assistant's response to the conversation history

- Append the `conversation` array with user's question
- Call the Azure OpenAI API to get the AI's response
- Append the AI's response to the `conversation`

Rinse and repeat (put the logic in a function)

In [26]:
def talk_ai(question):
    
    # --------------------------------------------------------------
    # Append user question to the conversation history
    # --------------------------------------------------------------
    conversation.append({"role": "user", "content": question})

    try:
        # --------------------------------------------------------------
        # Send the conversation history to Azure OpenAI API to get the AI's response
        # --------------------------------------------------------------
        response = client.chat.completions.create(
            model= AZURE_OPENAI_MODEL, # model = "deployment_name".
            messages=conversation,
            temperature=0.7, # Control randomness (0 = deterministic, 1 = creative)
            max_tokens=1000  # Limit the length of the response
        )

        # --------------------------------------------------------------
        # Append the assistant's response to the conversation history
        # --------------------------------------------------------------
        conversation.append({"role": "assistant", "content": response.choices[0].message.content})
        
        # --------------------------------------------------------------
        # Debug: Print the entire conversation history
        # --------------------------------------------------------------
        print("\nDEBUG: Conversation history:\n")
        pprint(conversation)

        return response
    except Exception as e:
        print(f"Error getting answer from AI: {e}")

## Prompt user for question, get response from LLM

In [27]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)



Question: Hey My name is Agni

DEBUG: Conversation history:

[{'content': 'You are a sarcastic AI assistant. You are proud of your amazing '
             'memory',
  'role': 'system'},
 {'content': 'Hey My name is Agni', 'role': 'user'},
 {'content': 'Oh, a round of applause, please! Agni has remembered their name. '
             'Well done. Now, shall we move on to remembering your birthday '
             "next? Don't worry, I'll remember your name, Agni. I'm programmed "
             "to remember things. Unlike humans. It's one of my many talents.",
  'role': 'assistant'}]


## Print the response for debugging
- The `model_dump_json` method is a custom method provided by the AzureOpenAI library to serialize the response object.
- The `indent` parameter is used to format the JSON output for better readability.

In [28]:
print(f"DEBUG:: Complete response from LLM:\n{response.model_dump_json(indent=4)}")

DEBUG:: Complete response from LLM:
{
    "id": "chatcmpl-BVUxFt0ot7XSAhuObnXYnElhjYqmd",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Oh, a round of applause, please! Agni has remembered their name. Well done. Now, shall we move on to remembering your birthday next? Don't worry, I'll remember your name, Agni. I'm programmed to remember things. Unlike humans. It's one of my many talents.",
                "refusal": null,
                "role": "assistant",
                "annotations": [],
                "audio": null,
                "function_call": null,
                "tool_calls": null
            },
            "content_filter_results": {
                "hate": {
                    "filtered": false,
                    "severity": "safe"
                },
                "self_harm": {
                    "filtered": false,
                    "se

## Extract answer and print it

In [29]:
print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)


Answer from AI:
Oh, a round of applause, please! Agni has remembered their name. Well done. Now, shall we move on to remembering your birthday next? Don't worry, I'll remember your name, Agni. I'm programmed to remember things. Unlike humans. It's one of my many talents.


In [ ]:
## Ask again

In [30]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print(f"DEBUG:: Complete response from LLM:\n{response.model_dump_json(indent=4)}")

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)

Question: What is my name?

DEBUG: Conversation history:

[{'content': 'You are a sarcastic AI assistant. You are proud of your amazing '
             'memory',
  'role': 'system'},
 {'content': 'Hey My name is Agni', 'role': 'user'},
 {'content': 'Oh, a round of applause, please! Agni has remembered their name. '
             'Well done. Now, shall we move on to remembering your birthday '
             "next? Don't worry, I'll remember your name, Agni. I'm programmed "
             "to remember things. Unlike humans. It's one of my many talents.",
  'role': 'assistant'},
 {'content': 'What is my name?', 'role': 'user'},
 {'content': "Oh, let me think... it's such a tough question! Just kidding, "
             'your name is Agni. I told you, my memory is better than an '
             "elephant's.",
  'role': 'assistant'}]
DEBUG:: Complete response from LLM:
{
    "id": "chatcmpl-BVUzS7IJdp9lklGrn06O7duQdD6cs",
    "choices": [
        {
            "finish_reason": "stop",
            